In [43]:
from lxml import etree 
import time 
import logging
import os
import sqlite3
import pathlib
import glob
import gzip 
import csv 

import pandas as pd
import xmltodict

DOSSIER_PARENT = "."
DOSSIER_SOURCE = "./todo/"
DOSSIER_SORTIE = "./done/"
BDD = 'bdd_actes_budgetaires.db'
FICHIER_CSV = 'donnees_lignes_budgets.csv'

way = "./todo/20200807-1825601.xml.gz"

FONCTIONS TIMING

In [30]:
def timing_decorator_x10x(func):
 def wrapper(*args, **kwargs):
  total_time = 0
  num_runs = 10
  for _ in range(num_runs):
   start_time = time.time()
   func(*args, **kwargs)
   end_time = time.time()
   total_time += end_time - start_time
   average_time = total_time / num_runs
   print(f"Temps moyen d'exécution de {func.__name__}: {average_time:.6f} secondes")
 return wrapper

def timing_decorator(func):
 def wrapper(*args, **kwargs):
  start_time = time.time()
  result = func(*args, **kwargs)
  end_time = time.time()
  execution_time = end_time - start_time
  print(f"{func.__name__} a pris {execution_time:.4f} secondes pour s'exécuter.")
  return result
 return wrapper

PARSE ET EXTRACT

In [31]:
def creation_racine(fichier):
 with gzip.open(fichier, 'rb') as fichier:
   arbre = etree.parse(fichier)
 racine = arbre.getroot()
 return racine

def _extract_metadonnees(racine, tag_name) -> dict: 
 result_dict = {}
 elements = racine.xpath(f"//*[contains(local-name(), '{tag_name}')]") #les tags sont tous précédés de {minefi/..../}, sert à ne pas les prendre en compte
 for element in elements:
  element_dict = {}
  for sous_element in element: 
    #Va chercher les valeurs liées aux clefs V
    element_dict[sous_element.tag] = sous_element.attrib.get("V", None) 
  result_dict.update(element_dict)
 return result_dict

def extract_ligne_budget(racine, tag_name) -> pd.DataFrame:
 ''' Recherche dans la racine spécifique au budget, à la différence des métadonnées, les lignes budgets sont nombreuses'''
 lignes_budget_list = []
 elements = racine.xpath(f"//*[contains(local-name(), '{tag_name}')]")
 for element in elements:
  ligne_budget_dict = {}
  for sous_element in element:
   ligne_budget_dict[sous_element.tag] = sous_element.attrib.get("V", None)
   lignes_budget_list.append(ligne_budget_dict) 
 df = pd.DataFrame(lignes_budget_list)
 df = df.rename(columns=lambda x: x.split('}')[-1])
 return df

def _extract_id(fichier) -> dict: 
 "Extrait l'id sous forme de dictionnaire pour le traitement"
 val_IdFichier = fichier.split("-")[-1].split('.')[0]
 IdFichier = {"IdFichier" : val_IdFichier}
 return IdFichier

def fusion_dicts_to_df(metadonnees_dict, id_dict) : 
 colonnes = ["IdFichier", "DteStr", "LibelleColl", "IdColl"]
 gros_dict = {**metadonnees_dict, **id_dict}
 df_metadonnees = pd.DataFrame(gros_dict, index = [0])
 df_metadonnees = df_metadonnees.rename(columns=lambda x: x.split('}')[-1])
 df_metadonnees = df_metadonnees[colonnes]
 return df_metadonnees

def extract_metadonnes(racine, fichier) -> pd.DataFrame :
 metadonnees_dict = _extract_metadonnees(racine, "EnTeteDocBudgetaire")
 id_dict =  _extract_id(fichier)
 metadonnees_df = fusion_dicts_to_df (metadonnees_dict, id_dict)
 return metadonnees_df

def assemblage_unitaire(df_lignes_budget, df_metadonnees) : 
 """ Assemble les dataFrame contenant les metadonnees et les lignes budgetaires """
 colonnes_a_conserver = ['IdFichier', 'Nomenclature', 'DteStr',
                         'LibelleColl', 'IdColl', 'Nature',
                         'LibCpte', 'Fonction', 'Operation',
                         'ContNat', 'ArtSpe', 'ContFon',
                         'ContOp', 'CodRD', 'MtBudgPrec',
                         'MtRARPrec', 'MtPropNouv', 'MtPrev',
                         'CredOuv', 'MtReal', 'MtRAR3112',
                         'OpBudg', 'MtSup_1_Lib', 'MtSup_1_Val',
                         'MtSup_2_Lib', 'MtSup_2_Val', 'CaracSup_Lib',
                         'CaracSup_Val', 'TypOpBudg', 'OpeCpteTiers' 
                        ]
 df_metadonnees = pd.concat([df_metadonnees] * len(df_lignes_budget), ignore_index=True)
 df_concat = pd.concat([df_metadonnees, df_lignes_budget], axis = 1)
 df_fichier = pd.DataFrame(columns=colonnes_a_conserver)

 for i in df_concat.columns : #Ne conserve que les colonnes qui nous intéressent,
    if i in colonnes_a_conserver :
        df_fichier[i] = df_concat[i]

 df_fichier = df_fichier.dropna(axis = 1, how = 'all')
 return df_fichier

def insertion_bdd(df_final: pd.DataFrame):
 """ insert dans une bdd les données maintenant transformées et en sort un csv à jour """
 chemin_bdd = os.path.join(DOSSIER_PARENT, BDD)
 conn = sqlite3.connect(chemin_bdd)
 df_final.to_sql('actes_budgetaire', conn,
                    if_exists='append', index=False)
 
def insertion_csv_methode_1(df_final : pd.DataFrame) :
 ''' S'ajoute à un csv déjà existant, permet de ne pas créer un csv à chaque fois, n'utilise pas la bdd '''
 with open(FICHIER_CSV, 'a', newline='') as fichier_csv : 
  ajout = csv.writer(fichier_csv)
  for _, ligne in df_final.iterrows():
   ajout.writerow(ligne)

SECURITE

In [32]:
def isolement_id(fichier) : 
 "Extrait l'id du nom du fichier pour la liste comprehension de securité"
 val_IdFichier = fichier.split("-")[-1].split('.')[0]
 return val_IdFichier

def recherche_id_dans_bdd():
    conn = sqlite3.connect(BDD)
    cursor = conn.cursor()
    cursor.execute(''' SELECT DISTINCT IdFichier FROM actes_budgetaire ''')
    ligne_sql_int = [int(x[0]) for x in cursor.fetchall()]
    conn.close()
    return ligne_sql_int

TEST

In [33]:
def clean_csv() : 
 with open(FICHIER_CSV, 'w', newline='') as csvfile:
  header = "IdFichier,DteStr,LibelleColl,IdColl,Nature,LibCpte,Fonction,Operation,ContNat,ArtSpe,ContFon,ContOp,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,TypOpBudg,OpeCpteTiers\n"
  csvfile.write(header)

def clean_bdd() :  
 conn = sqlite3.connect(BDD)
 cursor = conn.cursor()
 cursor.execute('''DELETE FROM actes_budgetaire ''')
 conn.commit()
 conn.close()

In [7]:
def ouverture_gzip(chemin) : 
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def parse_budget(data_dict: dict) -> pd.DataFrame : 
 "Sépare les sous clefs lignes budget, sans nettoyage"
 ligne_budget = data_dict['DocumentBudgetaire']['Budget']['LigneBudget']
 df_ligne_budget = pd.DataFrame(ligne_budget)
 return df_ligne_budget

def parse_metadonnees(data_dict : dict) -> pd.DataFrame : 
 "Sépare les sous clefs de métadonnées, sans nettoyage"
 metadonnees = data_dict['DocumentBudgetaire']['EnTeteDocBudgetaire']
 df_metadonnees = pd.DataFrame(metadonnees)
 return df_metadonnees

def parse_schema(data_dict : dict) -> pd.DataFrame : 
 "Sépare la version schema sans nettoyage"
 version_schema = data_dict['DocumentBudgetaire']['VersionSchema']['@V']
 df_schema = pd.DataFrame({"VersionSchema": [version_schema]})
 return df_schema

def parse_date(data_dict : dict) -> pd.DataFrame : 
 date = data_dict['DocumentBudgetaire']['Budget']['BlocBudget']
 df_date = pd.DataFrame(date)
 return df_date

def assemblage(df_principal: pd.DataFrame, 
                         df_meta: pd.DataFrame, 
                         df_date : pd.DataFrame) -> pd.DataFrame:
 """ Assemble les dataFrame contenant les metadonnees,
 la version schema et les lignes budgetaires """
 colonnes_a_conserver = ["IdFichier", "DteStr", "LibelleColl",
                         "IdColl","Nature","LibCpte",
                         "Fonction","Operation",
                         "ContNat","ArtSpe",
                         "ContFon", "ContOp",
                         "CodRD","MtBudgPrec",
                         "MtRARPrec","MtPropNouv",
                         "MtPrev","CredOuv",
                         "MtReal","MtRAR3112",
                         "OpBudg","TypOpBudg",
                         "OpeCpteTiers"
                        ]
 
 df_final = pd.DataFrame(columns=colonnes_a_conserver)
 df_meta = pd.concat([df_meta] * len(df_principal), ignore_index=True)
 df_date = pd.concat([df_date] * len(df_principal), ignore_index=True)

 for col in df_meta.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_meta[col]

 for col in df_principal.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_principal[col]

 for col in df_date.columns:
  if col in colonnes_a_conserver:
    df_final[col] = df_date[col]

 df_final = df_final.dropna(axis=1, how='all')
 return df_final

def nettoyage_lambda(df : pd.DataFrame) -> pd.DataFrame : 
 "Nettoie les données pour se débarasser des @V"
 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in df.columns : 
  df[col] = df[col].apply(nettoyage)
 return df 

def insertion_bdd(df_final: pd.DataFrame):
 """ insert dans une bdd les données maintenant transformées et en sort un csv à jour """
 chemin_bdd = os.path.join(DOSSIER_PARENT, BDD)
 conn = sqlite3.connect(chemin_bdd)
 df_final.to_sql('actes_budgetaire', conn,
                    if_exists='append', index=False)
 
def creation_csv() :
 conn = sqlite3.connect(BDD)
 cursor = conn.cursor()
 info = cursor.execute('''Select * from actes_budgetaire''')
 df = pd.DataFrame(info)
 fichier_csv = os.path.join(DOSSIER_PARENT, FICHIER_CSV)
 df.to_csv(fichier_csv, index=False)
 conn.close()

def deplacement_fichier(fichier_a_deplacer, dossier_destination):
 """ Déplace le fichier du dossier source au dossier fini"""
 chemin_source = pathlib.Path(fichier_a_deplacer)
 chemin_destination = pathlib.Path(dossier_destination) / chemin_source.name
 chemin_source.rename(chemin_destination)

def main():
 """ Traitement global, extrait et transforme les fichiers XML dans DOSSIER_SOURCE
    pour les insérer dans une bdd et en faire un csv"""
 liste_des_fichier_source = glob.glob(os.path.join(DOSSIER_SOURCE, "*.gz"))
 liste_des_fichier_traite = glob.glob(os.path.join(DOSSIER_SORTIE, "*.gz"))
 liste_des_df = []
 for fichier in liste_des_fichier_source:
  logging.info(f'Debut du travail sur {fichier}')
    # Sécurité permettant de ne pas injecter des doublons
  if fichier in liste_des_fichier_traite : 
   logging.error(f'Le fichier {fichier} a déjà été traité')
   return None 
  else : 
   data_dict = ouverture_gzip(fichier)
   if data_dict is not None:
    df_budget = parse_budget(data_dict)
    df_metadonnees = parse_metadonnees(data_dict)
    df_date = parse_date(data_dict)
    if df_budget is not None \
            and df_metadonnees is not None \
            and df_date is not None:
     df_final = assemblage(
                df_budget, df_metadonnees, df_date)
     liste_des_df.append(df_final)
     logging.info(f'Fin du travail sur {fichier}')
 df_session = pd.concat(liste_des_df, ignore_index = True)
 nettoyage_lambda(df_session)
 insertion_bdd(df_session)
 insertion_csv(df_session)
 clean_bdd()
 clean_csv()


In [10]:
FICHIERS_TODO = glob.glob(os.path.join(DOSSIER_SOURCE, "*.gz"))
FICHIERS_SAFE = [fichier for fichier in FICHIERS_TODO if int(isolement_id(fichier)) not in recherche_id_dans_bdd()]

In [35]:
def _1extract_ligne_budget(racine, tag_name) -> pd.DataFrame:
 lignes_budget_list = []
 elements = racine.xpath(f"//*[contains(local-name(), '{tag_name}')]")
 for element in elements:
        ligne_budget_dict = {}
        for sous_element in element:
            ligne_budget_dict[sous_element.tag] = sous_element.attrib.get("V", None)
        lignes_budget_list.append(ligne_budget_dict) 
 df = pd.DataFrame(lignes_budget_list)
 df = df.rename(columns=lambda x: x.split('}')[-1])
 return df

In [44]:
racine1 = creation_racine(way)
meta1 = _extract_metadonnees(racine1, "EnTeteDocBudgetaire")
budget1 = _1extract_ligne_budget(racine1, 'LigneBudget')
id_dict1 = _extract_id(way)
dict_fusionne = fusion_dicts_to_df(meta1, id_dict1) 
budget1.head(5)

,Nature,Fonction,ArtSpe,CodRD,MtPrev,CredOuv,OpBudg,TypOpBudg,CaracSup,MtReal,MtSup,MtRARPrec,MtRAR3112,LibCpte,OpeCpteTiers,ContNat
0,023,01,false,D,8127884.82,8127884.82,1,2,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6042,324,false,D,1000.00,1000.00,0,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60611,020,false,D,18760.00,18760.00,0,NaN,F,10123.97,NaN,NaN,NaN,NaN,NaN,NaN
3,60611,311,false,D,2710.00,2710.00,0,NaN,F,1599.10,NaN,NaN,NaN,NaN,NaN,NaN
4,60611,321,false,D,1565.00,1565.00,0,NaN,F,1069.20,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
@timing_decorator
def main_unitaire(objet) : 
 racine = creation_racine(objet)
 meta = _extract_metadonnees(racine, "EnTeteDocBudgetaire")
 budget = _1extract_ligne_budget(racine, "LigneBudget")
 id_dict = _extract_id(objet)
 metaa = fusion_dicts_to_df(meta, id_dict)
 df_1 = assemblage_unitaire(budget, metaa)
 insertion_bdd(df_1)
 insertion_csv(df_1)
 return df_1

main_unitaire(way)


OperationalError: table actes_budgetaire has no column named MtRARPrec

In [50]:
main()

insertion_csv a pris 0.8047 secondes pour s'exécuter.
Temps moyen d'exécution de main: 0.675610 secondes
insertion_csv a pris 0.8258 secondes pour s'exécuter.
Temps moyen d'exécution de main: 1.366826 secondes
insertion_csv a pris 0.7937 secondes pour s'exécuter.
Temps moyen d'exécution de main: 2.051272 secondes
insertion_csv a pris 0.8137 secondes pour s'exécuter.
Temps moyen d'exécution de main: 2.722986 secondes
insertion_csv a pris 0.7917 secondes pour s'exécuter.
Temps moyen d'exécution de main: 3.399803 secondes
insertion_csv a pris 0.8237 secondes pour s'exécuter.
Temps moyen d'exécution de main: 4.090873 secondes
insertion_csv a pris 0.8358 secondes pour s'exécuter.
Temps moyen d'exécution de main: 4.766069 secondes
insertion_csv a pris 0.8007 secondes pour s'exécuter.
Temps moyen d'exécution de main: 5.454139 secondes
insertion_csv a pris 0.8097 secondes pour s'exécuter.
Temps moyen d'exécution de main: 6.131662 secondes
insertion_csv a pris 0.8127 secondes pour s'exécuter.
T

In [39]:
@timing_decorator
def main_ancien(objet) :
   data_dict = ouverture_gzip(objet)
   df_budget = parse_budget(data_dict)
   df_metadonnees = parse_metadonnees(data_dict)
   df_date = parse_date(data_dict) 
   df_final = assemblage(
                df_budget, df_metadonnees, df_date)
   return df_final
 
@timing_decorator
def boucle_main_test() : 
  fichiers_safe = [fichier for fichier in FICHIERS_TODO if int(isolement_id(fichier)) not in recherche_id_dans_bdd()]
  for fichier in fichiers_safe : 
   main_unitaire(fichier)
   
#racine = creation_racine(way)
#meta = extract_metadonnees(racine, "EnTeteDocBudgetaire")
#budget = extract_ligne_budget(racine, "LigneBudget")
#id_dict = extract_id(way)
#date = extract_metadonnees(racine, "BlocBudget")
#metaa = fusion_dicts_to_df(meta, date, id_dict)
#df_1 = assemblage(budget, metaa)
#insertion_bdd(df_1)
#insertion_csv(df_1)

In [41]:
clean_csv()
clean_bdd()

In [40]:
boucle_main_test()
 
 

main_unitaire a pris 0.6051 secondes pour s'exécuter.


OperationalError: table actes_budgetaire has no column named MtRARPrec